In [1]:
# !pip install scikit-image
import pandas as pd
from skimage.feature import hog
import glob,os,cv2 as cv
import numpy as np 

In [2]:
train_data =[]
train_labels =[]

In [3]:
person = ['001','002','003','004','005','006','007','008','009','010']
orientation_list = ['nm-01','nm-02','nm-03','nm-04','nm-05','nm-06']
# list = ['000','018','036','054','072','090','108','126','144','162','180']

for per in person:
    for orl in orientation_list:
        folderPath = f"C:\\Users\\mritu\\OneDrive\\Desktop\\MAJOR PROJECT\\b_tech\\b_tech\\{per}\\{orl}"
        if not os.path.exists(folderPath):
            continue
           
        basePath = f"{folderPath}\\GENI_new"
        ext1 = "\\*.png"
        file1 = f"{basePath}{ext1}"
        imgPath = glob.glob(file1)
        images = [cv.imread(image,0) for image in imgPath]

        for im in images:
#             fd = cv.imread(im,0)
#           fd = cv2.resize(fd,(240,240))
            fd = hog(im,orientations=9,pixels_per_cell=(8,8),visualize=False,cells_per_block=(2,2))
            train_data.append(fd)
            train_labels.append(per)

           
          
#            path = f"C:\\Users\\mritu\\OneDrive\\Desktop\\MAJOR PROJECT\\b_tech\\b_tech\\{per}\\{orl}\\GENI_new\\{imName}"
#  imName = f"GENI_{per}_{orl}_{i}.png"
#            cv.imwrite(path,res)

In [4]:
test_data = []
test_labels = []
person = ['001','002','003','004','005','006','007','008','009','010']
orientation_list = ['bg-01','bg-02','cl-01','cl-02']
# list = ['000','018','036','054','072','090','108','126','144','162','180']

for per in person:
    for orl in orientation_list:
        folderPath = f"C:\\Users\\mritu\\OneDrive\\Desktop\\MAJOR PROJECT\\b_tech\\b_tech\\{per}\\{orl}"
        if not os.path.exists(folderPath):
            continue
           
        basePath = f"{folderPath}\\GENI_new"
        ext1 = "\\*.png"
        file1 = f"{basePath}{ext1}"
        imgPath = glob.glob(file1)
        images = [cv.imread(image,0) for image in imgPath]

        for im in images:
            fd = hog(im,orientations=9,pixels_per_cell=(8,8),visualize=False,cells_per_block=(2,2))
            test_data.append(fd)
            test_labels.append(per)

In [5]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
train_data.shape

(655, 30276)

In [6]:
test_data = np.array(test_data)
test_labels = np.array(test_labels)
test_data.shape

(427, 30276)

In [7]:
data = np.concatenate((train_data, test_data), axis=0)
label = np.append(train_labels, test_labels)
print(data.shape)
print(label.shape)

(1082, 30276)
(1082,)


In [8]:
from sklearn.decomposition import PCA
pca1 = PCA(n_components=200)
principalComponents = pca1.fit_transform(data)
principal_df = pd.DataFrame(data = principalComponents, columns = np.arange(1,201).astype(str))

In [9]:
print(principal_df.shape)

(1082, 200)


In [10]:
import pandas_profiling

In [11]:
# principal_df.profile_report()

In [12]:
train_data.shape

(655, 30276)

In [13]:
test_data.shape

(427, 30276)

In [14]:
# 'bg-01','bg-02',
from sklearn.model_selection import GridSearchCV, train_test_split

In [16]:
X_train = principal_df.iloc[:655,]
X_test = principal_df.iloc[655:,]
print(X_train.shape)
print(X_test.shape)

(655, 200)
(427, 200)


In [17]:
from sklearn.svm import LinearSVC
from sklearn import svm, metrics
from sklearn.preprocessing import LabelEncoder

In [19]:
model = LinearSVC()
print('Training...... Support Vector Machine')
model.fit(X_train,train_labels)

Training...... Support Vector Machine


LinearSVC()

In [20]:
pred = model.predict(X_test)

In [21]:
metrics.accuracy_score(pred,test_labels)

0.8126463700234192

In [43]:
metrics.precision_score(pred,test_labels,average="macro", sample_weight=None)

0.8162756598240468

In [44]:
metrics.recall_score(pred,test_labels,average="macro", sample_weight=None)

0.8320036683854755

In [45]:
metrics.f1_score(pred,test_labels,average="macro", sample_weight=None)

0.8151612448122745

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
rf = RandomForestClassifier(criterion='entropy')   
rf.fit(X_train,train_labels)

RandomForestClassifier(criterion='entropy')

In [24]:
pred2 = rf.predict(X_test)

In [25]:
metrics.accuracy_score(pred2,test_labels)

0.7892271662763466

In [34]:
metrics.precision_score(pred2,test_labels,average="macro", sample_weight=None)

0.7954545454545453

In [35]:
metrics.recall_score(pred2,test_labels,average="macro", sample_weight=None)

0.8095106009947755

In [40]:
metrics.f1_score(pred2,test_labels,average="macro", sample_weight=None)

0.7915457717364183

In [26]:
from sklearn.neighbors import KNeighborsClassifier

In [46]:
KNN = KNeighborsClassifier(n_neighbors = 5)
KNN.fit(X_train,train_labels)
pred3 = KNN.predict(X_test)
accuracy = metrics.accuracy_score(pred3,test_labels)
precision = metrics.precision_score(pred3,test_labels,average="macro", sample_weight=None)
recall = metrics.recall_score(pred3,test_labels,average="macro", sample_weight=None)
f1_score = metrics.f1_score(pred3,test_labels,average="macro", sample_weight=None)
print(accuracy)
print(precision)
print(recall)
print(f1_score)

0.8618266978922716
0.8659090909090909
0.8788372589978838
0.8607180327398712


In [28]:
# KNN = KNeighborsClassifier(n_neighbors = 10)
# KNN.fit(X_train,train_labels)

In [29]:
# pred3 = KNN.predict(X_test)

In [30]:
# metrics.accuracy_score(pred3,test_labels)